In [1]:
import os
import cv2
import tensorflow as tf
import numpy as np

CATEGORIES = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]

pathDataset = "C:/Users/user/Documents/Code/Machine-Learning/my_training_dataset"
path = "C:/Users/user/Documents/Code/Machine-Learning/imgToCharacters"
pathBuffer = "C:/Users/user/Documents/Code/Machine-Learning/bufferImg"

model = tf.keras.models.load_model("readable-model.model")

In [2]:
list_x  = []
list_img = []

IMSIZE = 50

for img in os.listdir(path):
    img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
    ret, thresh = cv2.threshold(img_array, 127, 255, 0)
    image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        
    for i in range(len(contours)):

        cnt = contours[i]
        x,y,w,h = cv2.boundingRect(cnt)
        if w*h > 30*40 and w*h < 200*200:
            img_crop = thresh[y:y+h,x:x+w]
            new_img = cv2.resize(img_crop, (IMSIZE, IMSIZE))
            list_img.append(new_img)
            list_x.append(x)
            cv2.imwrite(os.path.join(pathBuffer,"img%s.png" %i),img_crop)          

    
list_img = np.array(list_img).reshape(-1, 50, 50, 1)

print(len(list_img))

5


In [3]:
import pickle
img_array = pickle.load(open("characters_img.pickle", "rb"))
x_img = pickle.load(open("characters_x.pickle","rb"))

list_img = list_img / 255.0

prediction = model.predict(list_img)

In [4]:
array_x_char = []
for i in range(len(x_img)):
    array = [list_x[i], CATEGORIES[np.argmax(prediction[i])]]
    array_x_char.append(array)

print(array_x_char)    
array_x_char.sort(key=lambda x:x[0])

[[446, 'u'], [232, 'a'], [108, 's'], [383, 'l'], [517, 'k']]


In [5]:
text = []
for x, char in array_x_char:
    text.append(char)
    
print(text)

['s', 'a', 'l', 'u', 'k']


In [6]:
real_text = ["s","a","l","u","t"]

In [7]:
miss = []

for i in range(len(text)):
    if text[i] != real_text[i]:
        var = array_x_char[i][0]
        for x in range(len(list_x)):
            if var == list_x[x]:
                miss.append( [list_img[x] * 255.0, real_text[i] ] )

In [8]:
from random import random, seed
from datetime import datetime

seed(datetime.now())

for img, cat in miss:
    t = random()
    pathCat = os.path.join(pathDataset, cat)
    cv2.imwrite(os.path.join(pathCat,"img%s.png"%t, ), img)